## Import libraries

In [1]:
from selenium import webdriver 
from selenium.webdriver.firefox.options import Options 
from bs4 import BeautifulSoup 
import time 
import pandas as pd

## Company Profile

In [16]:
# company profile scraping (output as a list)

def get_profile(profile_list, company_nickname): 
    options = Options() 
    options.headless = True 
    driver = webdriver.Firefox()
    url = "https://www.wsj.com/market-data/quotes/{}/company-people".format(company_nickname)
    driver.get(url) 
    source = driver.page_source 
    soup = BeautifulSoup(source, "lxml") 
    driver.close() 
    name = soup.find("span", {"class": "hdr_co_name"}).text 
    holder = soup.find("div", {"class": "cr_data"}) 
    holder = holder.find_all("span", {"class": "data_data"}) 
    sector_part = holder[1].text 
    sector = sector_part.strip() 
    industry_part = holder[3].text 
    industry = industry_part.strip() 
    description_part = soup.find("p", {"class": "txtBody"}).text 
    description = description_part.strip() 
    profile_list.append(name) 
    profile_list.append(sector) 
    profile_list.append(industry) 
    profile_list.append(description)
    
profile_list = []    
get_profile(profile_list,'AMZN')
profile_list

['Amazon.com Inc.',
 'Mixed Retailing',
 'Retail/Wholesale',
 'Amazon.com, Inc. engages in the provision of online retail shopping services. It operates through the following business segments: North America, International, and Amazon Web Services (AWS). The North America segment includes retail sales of consumer products and subscriptions through North America-focused websites such as www.amazon.com and www.amazon.ca. The International segment offers retail sales of consumer products and subscriptions through internationally-focused websites. The Amazon Web Services segment involves in the global sales of compute, storage, database, and AWS service offerings for start-ups, enterprises, government agencies, and academic institutions. The company was founded by Jeffrey P. Bezos in July 1994 and is headquartered in Seattle, WA.']

In [17]:
import pandas as pd
profile_info = ['Company Info','Sector','Industry','Description']
company_profile_df = pd.DataFrame([profile_list], columns = profile_info)

company_profile_df.transpose()

,0
Company Info,Amazon.com Inc.
Sector,Mixed Retailing
Industry,Retail/Wholesale
Description,"Amazon.com, Inc. engages in the provision of o..."


## Company stock

In [2]:
# company's stock scraping (output as a list of rows)

def get_stock(stock_list, company_nickname): 
    options = Options() 
    options.headless = True 
    driver = webdriver.Firefox() 
    url = "https://www.wsj.com/market-data/quotes/{}/company-people".format(company_nickname)
    driver.get(url)
    textbox1 = driver.find_element_by_id("selectDateFrom") 
    textbox1.clear() 
    textbox1.send_keys('1/1/2015') 
    textbox2 = driver.find_element_by_id("selectDateTo") 
    textbox2.clear() 
    textbox2.send_keys('12/31/2019') 
    go_button = driver.find_element_by_id('datPickerButton') 
    driver.execute_script("arguments[0].click();", go_button) 
    time.sleep(10) 
    source = driver.page_source 
    soup = BeautifulSoup(source, "lxml") 
    table1 = soup("table", {"class": "cr_dataTable"})[0].find_all('tr') 
    for row in table1: 
        header = row.findChildren(recursive=False) 
        header = [ele.text.strip() for ele in header] 
        stock_list.append(header) 
    table2 = soup("table", {"class": "cr_dataTable"})[1].find_all('tr') 
    for row in table2: 
        values = row.findChildren(recursive=False) 
        values = [ele.text.strip() for ele in values] 
        stock_list.append(values) 
    driver.close() 

stock_list = []
get_stock(stock_list,'AMZN')
stock_list[:5]

KeyboardInterrupt: 

In [33]:
# putting company stock table into pandas dataframe

import pandas as pd
company_stock_df = pd.DataFrame(stock_list[1:], columns= stock_list[0])

company_stock_df.head()

,DATE,OPEN,HIGH,LOW,CLOSE,VOLUME
0,12/31/19,203.9950,205.5600,203.6001,205.2500,8.96 M
1,12/30/19,207.8600,207.9000,203.9001,204.4100,10.53 M
2,12/27/19,208.6697,208.9300,206.5875,208.1000,10.29 M
3,12/26/19,205.5670,207.8200,205.3100,207.7900,9.37 M
4,12/24/19,206.3000,206.7900,205.0000,205.1200,6.05 M


## Income statement

In [4]:
# income statement scraping (output as a list of rows)

def get_income_statement(list_for_dataframe, company_nickname): 
    options = Options() 
    options.headless = True 
    driver = webdriver.Firefox() 
    url = "https://www.wsj.com/market-data/quotes/{}/company-people".format(company_nickname)
    driver.get(url)
    source = driver.page_source 
    soup = BeautifulSoup(source, "lxml") 
    table = soup("table", {"class": "cr_dataTable"})[0].find_all('tr') 
    list1 = ['', '', '', '', ''] 
    list2 = [] 
    for row in table: 
        header = row.findChildren(recursive=False) 
        header = [ele.text.strip() for ele in header] 
        if header == list1 or header == list2: 
            continue 
        else: 
            header.pop() 
        list_for_dataframe.append(header) 
    driver.close() 

df_list = []
get_income_statement(df_list,'AMZN')
df_list[:3]

[['Name/Title']]

In [34]:
# putting income statement table into pandas dataframe

import pandas as pd
income_statement_df = pd.DataFrame(df_list[1:], columns= df_list[0])

income_statement_df.head()


,Fiscal year is January-December. All values USD Millions.,2019,2018,2017,2016,2015
0,Sales/Revenue,"70,697","55,838","40,653","27,638","17,928"
1,Sales Growth,26.61%,37.35%,47.09%,54.16%,-
2,Cost of Goods Sold (COGS) incl. D&A,"12,770","9,355","5,454","3,789","2,867"
3,COGS excluding D&A,"7,029","5,040","2,429","1,447",922
4,Depreciation & Amortization Expense,"5,741","4,315","3,025","2,342","1,945"


In [5]:
# income_statement_df.to_excel("C:\\Users\\Jason\\Documents\\CS109\\WebScraping\\kuihho_volume1.xlsx",index=False)


## Balance sheet

In [11]:
# balance sheet scraping (output as a list of rows)

def get_balance_sheet(list_for_dataframe, company_nickname): 
    options = Options() 
    options.headless = True 
    driver = webdriver.Firefox() 
    url = "https://www.wsj.com/market-data/quotes/{}/company-people".format(company_nickname)
    driver.get(url) 
    source = driver.page_source 
    soup = BeautifulSoup(source, "lxml") 
    num = [0, 1] 
    for x in num: 
        table = soup("table", {"class": "cr_dataTable"})[x].find_all('tr') 
        list1 = ['', '', '', '', ''] 
        list2 = [] 
        for row in table: 
            header = row.findChildren(recursive=False) 
            header = [ele.text.strip() for ele in header] 
            if header == list1 or header == list2: 
                continue 
            else: 
                header.pop() 
            list_for_dataframe.append(header)
    driver.close() 
    
    
bs_list = []
get_balance_sheet(bs_list,'AMZN')
bs_list

[['Fiscal year is January-December. All values USD Millions.',
  '2019',
  '2018',
  '2017',
  '2016',
  '2015'],
 ['Net Income before Extraordinaries', '-', '-', '-', '-', '-'],
 ['Cash & Short Term Investments',
  '54,863',
  '41,124',
  '41,711',
  '29,449',
  '18,434'],
 ['Cash Only', '19,087', '10,029', '8,079', '8,903', '4,907'],
 ['Short-Term Investments', '35,776', '31,095', '33,632', '20,546', '13,527'],
 ['Cash & Short Term Investments Growth',
  '33.41%',
  '-1.41%',
  '41.64%',
  '59.75%',
  '-'],
 ['Cash & ST Investments / Total Assets',
  '41.13%',
  '42.25%',
  '49.35%',
  '45.33%',
  '37.31%'],
 ['Total Accounts Receivable', '9,518', '7,587', '5,832', '3,993', '2,559'],
 ['Accounts Receivables, Net', '9,518', '7,587', '5,832', '3,993', '2,559'],
 ['Accounts Receivables, Gross', '9,724', '7,816', '6,021', '4,087', '2,627'],
 ['Bad Debt/Doubtful Accounts', '(206)', '(229)', '(189)', '(94)', '(68)'],
 ['Other Receivables', '-', '-', '-', '-', '-'],
 ['Accounts Receivable G

In [35]:
# putting balance sheet table into pandas dataframe

import pandas as pd
balance_sheet_df = pd.DataFrame(bs_list[1:], columns= bs_list[0])

balance_sheet_df.head()

,Fiscal year is January-December. All values USD Millions.,2019,2018,2017,2016,2015
0,Net Income before Extraordinaries,-,-,-,-,-
1,Cash & Short Term Investments,"54,863","41,124","41,711","29,449","18,434"
2,Cash Only,"19,087","10,029","8,079","8,903","4,907"
3,Short-Term Investments,"35,776","31,095","33,632","20,546","13,527"
4,Cash & Short Term Investments Growth,33.41%,-1.41%,41.64%,59.75%,-


## Cash flow

In [12]:
# cash flow scraping (output as a list of rows)

def get_cash_flow(list_for_dataframe, company_nickname): 
    options = Options() 
    options.headless = True 
    driver = webdriver.Firefox() 
    url = "https://www.wsj.com/market-data/quotes/{}/company-people".format(company_nickname)
    driver.get(url)
    source = driver.page_source 
    soup = BeautifulSoup(source, "lxml") 
    num = [0, 1, 2] 
    for x in num: 
        table = soup("table", {"class": "cr_dataTable"})[x].find_all('tr') 
        list1 = ['', '', '', '', ''] 
        list2 = [] 
        for row in table: 
            header = row.findChildren(recursive=False) 
            header = [ele.text.strip() for ele in header] 
            if header == list1 or header == list2: 
                continue 
            else: 
                header.pop() 
            list_for_dataframe.append(header) 
    driver.close() 
    
    
cf_list = []
get_cash_flow(cf_list,'AMZN')
cf_list

[['Fiscal year is January-December. All values USD Millions.',
  '2019',
  '2018',
  '2017',
  '2016',
  '2015'],
 ['Net Income before Extraordinaries',
  '18,485',
  '22,112',
  '15,934',
  '10,217',
  '3,688'],
 ['Net Income Growth', '-16.40%', '38.77%', '55.96%', '177.03%', '-'],
 ['Depreciation, Depletion & Amortization',
  '5,741',
  '4,315',
  '3,025',
  '2,342',
  '1,945'],
 ['Depreciation and Depletion', '5,179', '3,675', '2,333', '1,591', '1,215'],
 ['Amortization of Intangible Assets', '562', '640', '692', '751', '730'],
 ['Deferred Taxes & Investment Tax Credit',
  '(37)',
  '286',
  '(377)',
  '(457)',
  '(795)'],
 ['Deferred Taxes', '(37)', '286', '(377)', '(457)', '(795)'],
 ['Investment Tax Credit', '-', '-', '-', '-', '-'],
 ['Other Funds', '4,875', '4,088', '3,747', '3,248', '2,977'],
 ['Funds from Operations', '29,064', '30,801', '22,329', '15,350', '7,815'],
 ['Extraordinaries', '-', '-', '-', '-', '-'],
 ['Changes in Working Capital', '7,250', '(1,527)', '1,887', '7

In [36]:
# putting cash flow table into pandas dataframe
import pandas as pd
cash_flow_df = pd.DataFrame(cf_list[1:], columns= cf_list[0])

cash_flow_df.head()

,Fiscal year is January-December. All values USD Millions.,2019,2018,2017,2016,2015
0,Net Income before Extraordinaries,"18,485","22,112","15,934","10,217","3,688"
1,Net Income Growth,-16.40%,38.77%,55.96%,177.03%,-
2,"Depreciation, Depletion & Amortization","5,741","4,315","3,025","2,342","1,945"
3,Depreciation and Depletion,"5,179","3,675","2,333","1,591","1,215"
4,Amortization of Intangible Assets,562,640,692,751,730


## Exporting to Excel

In [38]:
# Doing it this way actually overwrites the previous excel exports

''' 
company_profile_df.to_excel("C:\\Users\\Jason\\Documents\\CS109\\WebScraping\\kuihho_volume1.xlsx",index=False,sheet_name='Facebook',startrow=2, startcol=0)

company_stock_df.to_excel("C:\\Users\\Jason\\Documents\\CS109\\WebScraping\\kuihho_volume1.xlsx",index=False,sheet_name='Facebook',startrow=9, startcol=0)

income_statement_df.to_excel("C:\\Users\\Jason\\Documents\\CS109\\WebScraping\\kuihho_volume1.xlsx",index=False,sheet_name='Facebook',startrow=9, startcol=10)

balance_sheet_df.to_excel("C:\\Users\\Jason\\Documents\\CS109\\WebScraping\\kuihho_volume1.xlsx",index=False,sheet_name='Facebook',startrow=9, startcol=20)

cash_flow_df.to_excel("C:\\Users\\Jason\\Documents\\CS109\\WebScraping\\kuihho_volume1.xlsx",index=False,sheet_name='Facebook',startrow=9, startcol=30)

'''

In [49]:
with pd.ExcelWriter("C:\\Users\\Jason\\Documents\\CS109\\WebScraping\\kuihho_volume1.xlsx", mode='w') as writer:
    company_profile_df.to_excel(writer,index=False, sheet_name='Facebook', startrow=2, startcol=0)
    company_stock_df.to_excel(writer,index=False, sheet_name='Facebook', startrow=9, startcol=0)
    income_statement_df.to_excel(writer,index=False, sheet_name='Facebook', startrow=9, startcol=10)
    balance_sheet_df.to_excel(writer,index=False, sheet_name='Facebook', startrow=9, startcol=20)
    cash_flow_df.to_excel(writer,index=False, sheet_name='Facebook', startrow=9, startcol=30)
    

I've encountered another problem here. Pandas can only export to a ordinary(non-macro enabled) excel sheet. That would mean a user would have to do the whole macro setting & recording again, incurring another manual step in the process. In light of that, I think the best workflow now is:

Python Script --> SQL Database (Schedule it to autorun the python script) --> Macro-enabled Excel Sheet (User just needs to click a button here which automatically import the data from SQL database and formats them into a presentable form with macros)

Things to do now:
1. Creating, managing the SQL database to collect new data
2. Schedule the database to run python scripts at a certain frequency
3. Learn VBA for Excel Automation





## Full blown process for each company

In [6]:
from selenium import webdriver 
from selenium.webdriver.firefox.options import Options 
from bs4 import BeautifulSoup 
import time 
import pandas as pd

In [10]:
def get_profile(profile_list, company_nickname): 
    options = Options() 
    options.headless = True 
    driver = webdriver.Firefox()
    url = "https://www.wsj.com/market-data/quotes/{}/company-people".format(company_nickname)
    driver.get(url) 
    source = driver.page_source 
    soup = BeautifulSoup(source, "lxml") 
    driver.close() 
    name = soup.find("span", {"class": "hdr_co_name"}).text 
    holder = soup.find("div", {"class": "cr_data"}) 
    holder = holder.find_all("span", {"class": "data_data"}) 
    sector_part = holder[1].text 
    sector = sector_part.strip() 
    industry_part = holder[3].text 
    industry = industry_part.strip() 
    description_part = soup.find("p", {"class": "txtBody"}).text 
    description = description_part.strip() 
    profile_list.append(name) 
    profile_list.append(sector) 
    profile_list.append(industry) 
    profile_list.append(description)
    
    
def get_stock(stock_list, company_nickname): 
    options = Options() 
    options.headless = True 
    driver = webdriver.Firefox() 
    url = "https://www.wsj.com/market-data/quotes/{}/historical-prices".format(company_nickname)
    driver.get(url)
    textbox1 = driver.find_element_by_id("selectDateFrom") 
    textbox1.clear() 
    textbox1.send_keys('1/1/2015') 
    textbox2 = driver.find_element_by_id("selectDateTo") 
    textbox2.clear() 
    textbox2.send_keys('12/31/2019') 
    go_button = driver.find_element_by_id('datPickerButton') 
    driver.execute_script("arguments[0].click();", go_button) 
    time.sleep(10) 
    source = driver.page_source 
    soup = BeautifulSoup(source, "lxml") 
    table1 = soup("table", {"class": "cr_dataTable"})[0].find_all('tr') 
    for row in table1: 
        header = row.findChildren(recursive=False) 
        header = [ele.text.strip() for ele in header] 
        stock_list.append(header) 
    table2 = soup("table", {"class": "cr_dataTable"})[1].find_all('tr') 
    for row in table2: 
        values = row.findChildren(recursive=False) 
        values = [ele.text.strip() for ele in values] 
        stock_list.append(values) 
    driver.close() 

    
def get_income_statement(list_for_dataframe, company_nickname): 
    options = Options() 
    options.headless = True 
    driver = webdriver.Firefox() 
    url = "https://www.wsj.com/market-data/quotes/{}/financials/annual/income-statement".format(company_nickname)
    driver.get(url)
    source = driver.page_source 
    soup = BeautifulSoup(source, "lxml") 
    table = soup("table", {"class": "cr_dataTable"})[0].find_all('tr') 
    list1 = ['', '', '', '', ''] 
    list2 = [] 
    for row in table: 
        header = row.findChildren(recursive=False) 
        header = [ele.text.strip() for ele in header] 
        if header == list1 or header == list2: 
            continue 
        else: 
            header.pop() 
        list_for_dataframe.append(header) 
    driver.close() 
    
    
def get_balance_sheet(list_for_dataframe, company_nickname): 
    options = Options() 
    options.headless = True 
    driver = webdriver.Firefox() 
    url = "https://www.wsj.com/market-data/quotes/{}/financials/annual/balance-sheet".format(company_nickname)
    driver.get(url) 
    source = driver.page_source 
    soup = BeautifulSoup(source, "lxml") 
    num = [0, 1] 
    for x in num: 
        table = soup("table", {"class": "cr_dataTable"})[x].find_all('tr') 
        list1 = ['', '', '', '', ''] 
        list2 = [] 
        for row in table: 
            header = row.findChildren(recursive=False) 
            header = [ele.text.strip() for ele in header] 
            if header == list1 or header == list2: 
                continue 
            else: 
                header.pop() 
            list_for_dataframe.append(header)
    driver.close()
    
    
def get_cash_flow(list_for_dataframe, company_nickname): 
    options = Options() 
    options.headless = True 
    driver = webdriver.Firefox() 
    url = "https://www.wsj.com/market-data/quotes/{}/financials/annual/cash-flow".format(company_nickname)
    driver.get(url)
    source = driver.page_source 
    soup = BeautifulSoup(source, "lxml") 
    num = [0, 1, 2] 
    for x in num: 
        table = soup("table", {"class": "cr_dataTable"})[x].find_all('tr') 
        list1 = ['', '', '', '', ''] 
        list2 = [] 
        for row in table: 
            header = row.findChildren(recursive=False) 
            header = [ele.text.strip() for ele in header] 
            if header == list1 or header == list2: 
                continue 
            else: 
                header.pop() 
            list_for_dataframe.append(header) 
    driver.close() 


In [20]:
company = {
            "FB": "Facebook", 
            "AMZN": "Amazon", 
            "AAPL": "Apple", 
            "NFLX": "Netflix", 
            "GOOG": "Google", 
            "MSFT": "Microsoft", 
            "TSLA": "Tesla", 
          }

for i in company:  
    print(i)
    profile_list = [] 
    stock_list = [] 
    is_list = []  
    bs_list = []
    cf_list = []
    
    get_profile(profile_list,i)
    get_stock(stock_list,i)
    get_income_statement(is_list,i)
    get_balance_sheet(bs_list,i)
    get_cash_flow(cf_list,i)
    
    profile_info = ['Company Info','Sector','Industry','Description']
    company_profile_df = pd.DataFrame([profile_list], columns = profile_info).transpose()
    company_stock_df = pd.DataFrame(stock_list[1:], columns= stock_list[0])
    income_statement_df = pd.DataFrame(is_list[1:], columns= is_list[0])
    balance_sheet_df = pd.DataFrame(bs_list[1:], columns= bs_list[0])
    cash_flow_df = pd.DataFrame(cf_list[1:], columns= cf_list[0])
    
    excel_workbook_name = "C:\\Users\\Jason\\Documents\\CS109\\WebScraping\\kuihhos_companies\\{}.xlsx".format(company[i])
    with pd.ExcelWriter(excel_workbook_name, mode='w') as writer:
        company_profile_df.to_excel(writer,index=False, sheet_name= company[i], startrow=2, startcol=0)
        company_stock_df.to_excel(writer,index=False, sheet_name=company[i], startrow=9, startcol=0)
        income_statement_df.to_excel(writer,index=False, sheet_name=company[i], startrow=9, startcol=10)
        balance_sheet_df.to_excel(writer,index=False, sheet_name=company[i], startrow=9, startcol=20)
        cash_flow_df.to_excel(writer,index=False, sheet_name=company[i], startrow=9, startcol=30)
    

FB
AMZN
AAPL
NFLX
GOOG
MSFT
TSLA
